In [ ]:
import import_ipynb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LassoCV
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split, GridSearchCV
from sklearn.metrics import auc, accuracy_score, plot_roc_curve

from math import cos,sqrt,pi

from show_chanWeights import show_chanWeights
from roc_util import plot_simple_roc

### Utility Functions
Functions that are called by **cross_validated_svm_result()** to generate the final output. Each function is further explained in its documentation.

In [7]:
def cross_validate_svm(X, y, train, test, search_space, ax=None, i=0):
    
    """This performs a 5-fold cross validation on input data, which is the second level of the 
    two-level cross-validation designed for this project
    
    Args:
        X: observation data as a (120, 204) matrix
        y: target data as a (120, ) vector
        train: indices for training data
        test: indices for testing data
        search_space: the range which we search for optimal hyperparameter in
        ax: plot axis to superimpose on
        i: the fold number of first level cross-validation
    
    Returns:
        acc: list of accuracies for each fold
        weights: list of (204, ) channel weights for each fold
        viz: a RocCurveDisplay object that stores computed values
    
    """
    
    # split data
    X_train, X_test = X[train, :], X[test, :]
    y_train, y_test = y[train], y[test]    
    
    # configure the cross-validation procedure
    cv_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
    # define the model
    model = SVC(kernel='linear', C=0.01, max_iter=1e5, random_state=42, probability=True)
    # define search space
    Cs = search_space
    # define search
    search = GridSearchCV(model, param_grid=dict(C=Cs), scoring='accuracy', n_jobs=-1, cv=cv_inner, refit=True)
    # execute search
    result = search.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate the model
    acc = best_model.score(X_test, y_test)  
    # get the weights for each channel
    weights = best_model.coef_[0]
    # return roc plot 
    viz = plot_roc_curve(best_model, X_test, y_test,
                     name='ROC fold {}'.format(i),
                     alpha=0.5, lw=1, ax=ax)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
    return acc, weights, viz

In [8]:
def plot_svm_roc(ax, tprs, aucs, mean_fpr):
    
    """Given a list of true positive rates and AUCs for each fold, plot a total ROC superimposed 
    on ROCs of all folds.
    
    Args:
        ax: plot axis to superimpose on
        tprs: list of true positive rates for each fold
        aucs: list of AUCs for each fold
        mean_fpr: a range of false positive rates
    
    """
    
    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
            label='Chance', alpha=.8)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    ax.plot(mean_fpr, mean_tpr, color='b',
            label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
            lw=2, alpha=.8)

    std_tpr = np.std(tprs, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                    label=r'$\pm$ 1 std. dev.')

    ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
           title="Receiver operating characteristic")
    ax.legend(loc="lower right")
    plt.show()    

In [9]:
def getKLargestWeights(arr, k):
    
    """Given an array, return a list of index-value pair for the largest k elements
    
    Args:
        arr: input array to sort
        k: number of largest elements to output
    
    Returns:
        A (k, 2) matrix that represents index-value pair of the k largest elements
    
    """
    
    # Sort the given array arr in reverse order.
    arr = np.array(arr)
    ind = np.abs(arr).argsort()[-5:].astype(int)
    val = arr[ind]
    return np.c_[ind, val]

def plot_weights(X, y, to_mark):
    
    """Plot the line graph based on the input X, y coordinates. Mark the elements of choice.
    
    Args:
        X: input range of X coordinate
        y: input Y value
        to_mark: specific coordinates that we want to mark on graph
    
    """
    
    plt.figure(figsize=(10, 4))
    markers_on = to_mark[:,0].astype(int)
    plt.plot(X, y, '-bD', markevery=np.array(markers_on))
    print('5 dominant channels:')
    for point in np.flip(to_mark, 0):
        plt.annotate("(%d, %.4f)" % (point[0], point[1]), point)
        print("channel %d (weight: %.4f)" % (point[0], point[1]))
    plt.title('Line Graph of weight value for each channel')
    plt.show()


## 1. Loading Input Data
We are given 4 sets of csv data, with 2 classes of each imagery motion data and overt motion data

In [2]:
img_1 = pd.read_csv('feaSubEImg_1.csv', header=None, dtype=float)
img_2 = pd.read_csv('feaSubEImg_2.csv', header=None, dtype=float)
ove_1 = pd.read_csv('feaSubEOvert_1.csv', header=None, dtype=float)
ove_2 = pd.read_csv('feaSubEOvert_2.csv', header=None, dtype=float)

## 2. Data Preprocessing
Each dataset (csv file) contains 120 observations, and each observation is composed of 204 features (channels). Here we transform these datas into a format that will be more easily passed to classifier model, and label them as 0 and 1 accordingly.

In [ ]:
# data pre processing
xImg_1 = img_1.transpose().to_numpy()
xImg_2 = img_2.transpose().to_numpy()
X_img = np.r_[xImg_1, xImg_2]

xOvert_1 = ove_1.transpose().to_numpy()
xOvert_2 = ove_2.transpose().to_numpy()
X_overt = np.r_[xOvert_1, xOvert_2]

y_1 = np.zeros(120)
y_2 = np.ones(120)
y = np.r_[y_1, y_2]

## 3. Two-Level Cross Validation
We implement two-level CV on the given dataset, keep track of the true positive rates and AUCs, and plot the necessary graphs as output.

In [10]:
def cross_validated_svm_result(X, y, search_space, display_channel=True, show_roc=True, show_weights=True):
    # Run classifier with cross-validation and plot ROC curves
    cv_outer = StratifiedKFold(n_splits=6, shuffle=True, random_state=0)

    tprs = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    # keep track of accuracy of each run
    outer_results = []
    # keep track of channel weights of each run
    weights_list = []

    fig, ax = plt.subplots(figsize=(7, 7))
    for i, (train, test) in enumerate(cv_outer.split(X, y)):
        # get all statistics for each run
        acc, weights, viz = cross_validate_svm(X, y, train, test, search_space, ax=ax, i=i)        
        # keep track of accuracy
        outer_results.append(acc)
        # append weights to weights list
        weights_list.append(weights)

        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)
    
    # summarize the estimated performance of the model
    print('Accuracy: %.3f (%.3f)' % (np.mean(outer_results), np.std(outer_results)))
    if show_roc:
        plot_svm_roc(ax, tprs, aucs, mean_fpr)
    if display_channel:
        show_chanWeights(np.abs(weights_list[0]))
    if show_weights:
        to_mark = getKLargestWeights(weights_list[0], 5)
        plot_weights(np.arange(1, 205), weights_list[0], to_mark=to_mark)